In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
import os
import shutil
import random

In [5]:
# Define paths
dataset_path = "Dataset/"
train_path = 'val/train'
validation_path = 'val/test'

In [6]:
# Create train and validation directories
if not os.path.exists(train_path):
    os.makedirs(train_path)
if not os.path.exists(validation_path):
    os.makedirs(validation_path)


In [7]:
# Split data into train and validation
for plant in os.listdir(dataset_path):
    plant_path = os.path.join(dataset_path, plant)
    if os.path.isdir(plant_path):
        images = os.listdir(plant_path)
        random.shuffle(images)
        train_size = int(0.8 * len(images))
        
        # Create class directories in train and validation folders
        train_class_path = os.path.join(train_path, plant)
        validation_class_path = os.path.join(validation_path, plant)
        
        if not os.path.exists(train_class_path):
            os.makedirs(train_class_path)
        if not os.path.exists(validation_class_path):
            os.makedirs(validation_class_path)
        
        for i, img in enumerate(images):
            img_path = os.path.join(plant_path, img)
            if os.path.isfile(img_path):
                if i < train_size:
                    shutil.copy(img_path, os.path.join(train_class_path, img))
                else:
                    shutil.copy(img_path, os.path.join(validation_class_path, img))
            else:
                print(f"File not found: {img_path}")

In [8]:
# Data generators
train_datagen = ImageDataGenerator(rescale=1./255)
validation_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_path,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical')

validation_generator = validation_datagen.flow_from_directory(
    validation_path,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical')

Found 1455 images belonging to 30 classes.
Found 380 images belonging to 30 classes.


In [9]:
# Debugging: Print directory structure
def print_directory_structure(root_dir):
    for root, dirs, files in os.walk(root_dir):
        level = root.replace(root_dir, '').count(os.sep)
        indent = ' ' * 4 * level
        print('{}{}/'.format(indent, os.path.basename(root)))
        subindent = ' ' * 4 * (level + 1)
        for f in files:
            print('{}{}'.format(subindent, f))

print("Train Directory Structure:")
print_directory_structure(train_path)

print("Validation Directory Structure:")
print_directory_structure(validation_path)

Train Directory Structure:
train/
    Alpinia Galanga (Rasna)/
        AG-S-001.jpg
        AG-S-002.jpg
        AG-S-003.jpg
        AG-S-004.jpg
        AG-S-005.jpg
        AG-S-006.jpg
        AG-S-008.jpg
        AG-S-009.jpg
        AG-S-010.jpg
        AG-S-012.jpg
        AG-S-014.jpg
        AG-S-015.jpg
        AG-S-016.jpg
        AG-S-017.jpg
        AG-S-018.jpg
        AG-S-019.jpg
        AG-S-020.jpg
        AG-S-021.jpg
        AG-S-022.jpg
        AG-S-023.jpg
        AG-S-028.jpg
        AG-S-029.jpg
        AG-S-030.jpg
        AG-S-031.jpg
        AG-S-032.jpg
        AG-S-033.jpg
        AG-S-034.jpg
        AG-S-035.jpg
        AG-S-036.jpg
        AG-S-037.jpg
        AG-S-038.jpg
        AG-S-039.jpg
        AG-S-041.jpg
        AG-S-043.jpg
        AG-S-045.jpg
        AG-S-046.jpg
        AG-S-047.jpg
        AG-S-048.jpg
        AG-S-049.jpg
        AG-S-050.jpg
    Amaranthus Viridis (Arive-Dantu)/
        AV-S-001.jpg
        AV-S-003.jpg
        AV-S-005.

In [10]:
# Data generators
train_datagen = ImageDataGenerator(rescale=1./255)
validation_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_path,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical')

validation_generator = validation_datagen.flow_from_directory(
    validation_path,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical')

print(f"Classes in training set: {train_generator.class_indices}")
print(f"Classes in validation set: {validation_generator.class_indices}")

Found 1455 images belonging to 30 classes.
Found 380 images belonging to 30 classes.
Classes in training set: {'Alpinia Galanga (Rasna)': 0, 'Amaranthus Viridis (Arive-Dantu)': 1, 'Artocarpus Heterophyllus (Jackfruit)': 2, 'Azadirachta Indica (Neem)': 3, 'Basella Alba (Basale)': 4, 'Brassica Juncea (Indian Mustard)': 5, 'Carissa Carandas (Karanda)': 6, 'Citrus Limon (Lemon)': 7, 'Ficus Auriculata (Roxburgh fig)': 8, 'Ficus Religiosa (Peepal Tree)': 9, 'Hibiscus Rosa-sinensis': 10, 'Jasminum (Jasmine)': 11, 'Mangifera Indica (Mango)': 12, 'Mentha (Mint)': 13, 'Moringa Oleifera (Drumstick)': 14, 'Muntingia Calabura (Jamaica Cherry-Gasagase)': 15, 'Murraya Koenigii (Curry)': 16, 'Nerium Oleander (Oleander)': 17, 'Nyctanthes Arbor-tristis (Parijata)': 18, 'Ocimum Tenuiflorum (Tulsi)': 19, 'Piper Betle (Betel)': 20, 'Plectranthus Amboinicus (Mexican Mint)': 21, 'Pongamia Pinnata (Indian Beech)': 22, 'Psidium Guajava (Guava)': 23, 'Punica Granatum (Pomegranate)': 24, 'Santalum Album (Sandalw

In [11]:
# Build the model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(len(train_generator.class_indices), activation='softmax')
])

c:\Users\Aniruddha\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [12]:
# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [13]:
# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=25,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size)

Epoch 1/25


c:\Users\Aniruddha\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


45/45 ━━━━━━━━━━━━━━━━━━━━ 57s 1s/step - accuracy: 0.0945 - loss: 3.7179 - val_accuracy: 0.4659 - val_loss: 1.9720
Epoch 2/25
 1/45 ━━━━━━━━━━━━━━━━━━━━ 12s 278ms/step - accuracy: 0.3333 - loss: 2.6359

c:\Users\Aniruddha\anaconda3\Lib\contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(value)


45/45 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.3333 - loss: 2.6359 - val_accuracy: 0.6071 - val_loss: 1.7731
Epoch 3/25
45/45 ━━━━━━━━━━━━━━━━━━━━ 42s 852ms/step - accuracy: 0.4647 - loss: 1.8530 - val_accuracy: 0.7159 - val_loss: 1.0442
Epoch 4/25
45/45 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.4000 - loss: 2.1286 - val_accuracy: 0.6071 - val_loss: 1.3253
Epoch 5/25
45/45 ━━━━━━━━━━━━━━━━━━━━ 43s 873ms/step - accuracy: 0.6695 - loss: 1.0985 - val_accuracy: 0.7642 - val_loss: 0.8281
Epoch 6/25
45/45 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.7188 - loss: 0.8856 - val_accuracy: 0.6071 - val_loss: 1.3151
Epoch 7/25
45/45 ━━━━━━━━━━━━━━━━━━━━ 44s 903ms/step - accuracy: 0.7755 - loss: 0.7576 - val_accuracy: 0.7926 - val_loss: 0.6670
Epoch 8/25
45/45 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.7500 - loss: 0.7373 - val_accuracy: 0.8929 - val_loss: 0.3627
Epoch 9/25
45/45 ━━━━━━━━━━━━━━━━━━━━ 44s 881ms/step - accuracy: 0.8360 - loss: 0.5019 - val_accuracy: 0.8381 - val_l

In [14]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 148, 148, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 74, 74, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 72, 72, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 36, 36, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 34, 34, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 17, 17, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 36992)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 512)            │    18,940,416 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 30)             │        15,390 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 57,147,164 (218.00 MB)

 Trainable params: 19,049,054 (72.67 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 38,098,110 (145.33 MB)

In [15]:
# Save the model
model.save('medicinal_plants_classifier.h5')

In [16]:
model.save('my_model.keras')